# Pneumonia Detection
By Atwine Mugume

In this kernel we are goin to apply `Transfer Learning`. 

You have been in that instance where you want to train a model but you don't have a GPU or enough RAM to help you do what you want to do. In this case you are put in a corner. 

Not anymore: you can use transfer learning. Transfer learning is the ability to take an already built model and use the weights of the model to train on your own data.

Just imagine a company like Google, has the resources to train a model over 4 million images for maybe 3 days, over 50+ classes.

With transfer learning you are able to take these models and work with them to solve your own problems.

How amazing is this?  Really amazing.

Let's get into it. 

We are going to be using a model called Resnet 50

You can read the paper on Resnet: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

##  Kaggle API

In order to get this data in your environment. You will have to download it from Kaggle.com.

You will need to download an authentication token, a json file which you will use in the setup process to work with their data.

In [1]:
#!pip install kaggle

## Kaggle Function:

I wrote this function below to help anyone automate the process of using their data from Kaggle.

What you will need:
1- A json file got from kaggle in the same folder as your other data

If you are working on colab you will have un comment out the lines that enable you to upload the json to the cloud platform.

However everything is done by the function below.

Just sit and watch

def kaggle_function():
  
  print('Please make sure you have downloaded a kaggle .jason token and its somewhere on your local machine')
  print('\n Let us first of all import the json token to Google Colab here')
  
  #first let's import the json file into the working directory
  #import json
  #from google.colab import files
  #files.upload()
  
  
  print('\n Alright sit back and let me handle the rest for you')
  
  #now that we have imported the kaggle json file let us make the directory needed to keep the json file
  !mkdir -p ~/.kaggle
  
  #let's copy the file there to the kaggle folder
  !cp kaggle.json ~/.kaggle/
  
  #let us encrypt the file so that we don't get hacked
  !chmod 600 ~/.kaggle/kaggle.json
  
  #now let's install kaggle packages
  #!pip install kaggle
  
  print("Success! Here is the Kaggle Competitions List to show this works")
  
  #let's do a simple listing of the files that kaggle has
  !kaggle competitions list
  
  print('\n\nWhat you need to do now is: go to the competition whose data you need and copy the download code and paste it in the next cell')

Now call the Kaggle function then sit tight and watch it go to work!!

In [2]:
#kaggle_function()

After the kaggle API is installed what is left is for you to go to kaggle.com and find the link from which you will download your data into your environment. Such as the one below.

Your data will be downloaded into your environment for you to use.

In [3]:
#let's download the dataset
#!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

#lets unzip the file
import zipfile

zipfilePath = ("./chest_xray.zip")
zip = zipfile.ZipFile(zipfilePath)
zip.extractall(".")
zip.close()

## Import the necessary libraries


In [4]:
import os
import sys
import glob
import matplotlib
import numpy as np
matplotlib.use('agg')
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from keras import backend as K
from keras import __version__
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from keras.layers import Dense, AveragePooling2D, GlobalAveragePooling2D, Input, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
from keras.preprocessing import image
from keras.models import load_model

Using TensorFlow backend.


## Data Visualisation Utility Functions

I need to work on some visualisations for the data that I have in my dataset

# Modeling

From here onwards we work on building the model that we are going to train to classify the images.

In [5]:
IM_WIDTH, IM_HEIGHT = 299, 299 
NB_EPOCHS = 20
BAT_SIZE = 32
FC_SIZE = 1024


%matplotlib inline

In [6]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [7]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [8]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = Dropout(0.5)(x)
    x = AveragePooling2D((8, 8), border_mode='valid', name='avg_pool')(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    predictions = Dense(2, activation='sigmoid')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [9]:
## For best Val ACC found, save the model at that epoch
filepath="Resnet50_weights_3.h5"
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
def train(epochs, output_model_file, plot = True ):
    """Using transfer learning """
    train_img = 'chest_xray/train/' 
    validation_img = 'chest_xray/val/'
    nb_epoch = int(epochs)
    nb_train_samples = get_nb_files(train_img)
    nb_classes = len(glob.glob(train_img + "/*"))
    # data prep
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    validation_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    
    train_generator = train_datagen.flow_from_directory(
			train_img,
			target_size=(299, 299),
			batch_size=32,
			class_mode='categorical'
			)
    validation_generator = validation_datagen.flow_from_directory(
			validation_img,
			target_size=(299, 299),
			batch_size=32,
			class_mode='categorical'
			)
    if(K.image_dim_ordering() == 'th'):
        input_tensor = Input(shape=(3, 299, 299))
    else:
        input_tensor = Input(shape=(299, 299, 3))
    
    # setup model
    from keras.applications.resnet50 import ResNet50
    base_model = ResNet50(input_tensor = input_tensor,weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    
    # transfer learning
    setup_to_transfer_learn(model, base_model)
    
    
    
    history_tl = model.fit_generator(train_generator,
                                   samples_per_epoch=320,
                                   nb_epoch=nb_epoch,
                                   validation_data=validation_generator,
                                   nb_val_samples=64,
                                   callbacks=[lr_reduce,checkpoint]) 
    model.save(output_model_file)
    if plot:
        plot_training(history_tl)

In [11]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')
    plt.savefig('accuracy.png')
    
    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
    plt.savefig('loss.png')

In [ ]:
train(epochs = 25, output_model_file = './resnet_new_model.h5')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D((8, 8), name="avg_pool", padding="valid")`
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically ste

Epoch 1/25
10/10 [==============================] - 667s 67s/step - loss: 0.5126 - acc: 0.7500 - val_loss: 2.0078 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to Resnet50_weights_3.h5
Epoch 2/25
10/10 [==============================] - 636s 64s/step - loss: 0.3396 - acc: 0.8344 - val_loss: 1.4296 - val_acc: 0.5000

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/25
10/10 [==============================] - 643s 64s/step - loss: 0.3209 - acc: 0.8313 - val_loss: 1.1648 - val_acc: 0.5000

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/25
10/10 [==============================] - 635s 63s/step - loss: 0.2761 - acc: 0.8781 - val_loss: 1.0357 - val_acc: 0.5000

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 00004: val_acc did not imp

### Validation accuracy achieved ~80%. 

## Predictions on test data

In [ ]:
target_size = (299, 299) # for Resnet50

In [ ]:
def predict(model, img, target_size = target_size):
    img = cv2.imread(img)
    img = cv2.resize(img,(299,299))
    x = np.reshape(img,[299,299,3])
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]

In [ ]:
def plot_preds(image, preds):
    labels = ("NORMAL", "PNEUMONIA")
    plt.barh([0, 1], preds, alpha=0.5)
    plt.yticks([0, 1], labels)
    plt.xlabel('Probabilities')
    plt.xlim(0,1.01)
    plt.tight_layout()

In [ ]:
# Load saved model
model = load_model('Resnet50_weights_3.h5')

### 0: Normal    1: Pneumonia

In [ ]:
# Pred 1
img =  'chest_xray/test/PNEUMONIA/person48_virus_100.jpeg'
preds = predict(model, img)
plot_preds(img, preds)

error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
res = np.argmax(preds)
print(res)   

In [ ]:
# Pred 2
img =  'chest_xray/test/NORMAL/IM-0105-0001.jpeg'
preds = predict(model, img)
plot_preds(img, preds)

In [ ]:
res = np.argmax(preds)
print(res)   

In [ ]:
# Pred 3
img =  'chest_xray/test/PNEUMONIA/person1672_virus_2888.jpeg'
preds = predict(model, img)
plot_preds(img, preds)

In [ ]:
res = np.argmax(preds)
print(res)   